# Intro

# Tích phân Monte Carlo 

Giả sử có $g(x)$ là hàm muốn tính $\int_{a}^{b} g(x) dx$  
$X$ là biến ngẫu nhiên có hàm mật độ $f(x)$  
$Y = g(X)$ 

Bước 1: Tính toán hàm kỳ vọng toán học $ E[g(X)] $ theo công thức:
$ E[g(X)] = \int_{-\infty}^{\infty} g(x) \cdot f(x)  dx $

Trong đó:
- $g(x)$  hàm số muốn tính kỳ vọng của nó.
- $f(x)$ là hàm mật độ xác suất của biến ngẫu nhiên $X$.

Bước 2: Nếu ta có một mẫu ngẫu nhiên từ phân phối của $ X $, ta có thể ước lượng không thiên vị của $ E[g(X)] $ bằng trung bình của mẫu. Cụ thể, nếu ta có một mẫu ngẫu nhiên $ x_1, x_2, ..., x_n $ từ phân phối của $ X $, thì ước lượng không thiên vị của $ E[g(X)] $ là:

$[ \hat{E}[g(X)] = \frac{1}{n} \sum_{i=1}^{n} g(x_i) ]$

Trong đó:
- $ \hat{E}[g(X)] $ là ước lượng không thiên vị của kỳ vọng toán học của $ Y $.
- $ g(x_i) $ là giá trị của hàm $ g $ tại mẫu thứ $ i $.

## Ước lượng tích phân Monte Calor đơn giản 

### Ví dụ 6.1  
Tính $\theta = \int_{0}^{1} e^{-x} dx$

In [1]:
m <- 10000
x <- runif(m)   # tạo theo phân phối đều 
theta.hat <- mean(exp(-x))  # tính kỳ vọng thông qua trung bình của tổng các giá trị f(x)
print(theta.hat)
print(1 - exp(-1))  # giá trị tích phân thực 

[1] 0.6307232
[1] 0.6321206


### Ví dụ 6.2  
Tính $\theta = \int_{2}^{4} e^{-x} dx$

In [3]:
m <- 10000
x <- runif(m, min = 2, max = 4) # tạo theo phân phối đều
theta.hat <- mean(exp(-x)) * (4 - 2) # tính kỳ vọng thông qua trung bình của tổng các giá trị f(x)
print(theta.hat)
print(exp(-2) - exp(-4)) # giá trị tích phân thực

[1] 0.1169846
[1] 0.1170196


...

# Importance Sampling

Làm lại ví dụ tính tích phân ở trên:  
Chọn hàm quan trọng $h(x)$  
Như vậy tích phân $\int_{0}^{\pi/3} sin(x) dx$ = $mean(\frac{f(x)}{h(x)})$

In [4]:
# Số lượng điểm được lấy mẫu
num_samples <- 100000

# Lấy mẫu ngẫu nhiên các giá trị của t từ 0 đến pi/3
t_values <- runif(num_samples, min = 0, max = pi/3)

# Tính giá trị của hàm sin(t) tại các giá trị của t
sin_values <- sin(t_values)

# Hàm quan trọng: h(t) = pi/3
h_values <- rep(pi/3, num_samples)  # (pi/3, pi/3, ..., pi/3)

# Tính tổng trung bình của các giá trị sin(t) / h(t)
monte_carlo_estimate <- mean(sin_values / h_values)

# Giá trị thực tế của tích phân
true_value <- 2/3

# In ra kết quả ước lượng Monte Carlo và giá trị thực tế
cat("Monte Carlo Estimate:", monte_carlo_estimate, "\n")
cat("True Value:", true_value, "\n")


Monte Carlo Estimate: 0.4559224 
True Value: 0.6666667 


## Ví dụ 6.11  
Tính $\int_{0}^{1} \frac{e^{-x}}{1 + x^2}dx$

In [5]:
m <- 10000
theta.hat <- se <- numeric(5)

g <- function(x) {
    exp(-x - log(1 + x^2)) * (x > 0) * (x < 1)
}

x <- runif(m)
# using f0
fg <- g(x) / 1 # 1 là hàm mật độ của hàm mẫu chia 
theta.hat[1] <- mean(fg) * (1 - 0)
se[1] <- sd(fg)

x <- rexp(m, 1)
# using f1
fg <- g(x) / exp(-x)
theta.hat[2] <- mean(fg) * (1 - 0)
se[2] <- sd(fg)

x <- rcauchy(m)
# using f2
i <- c(which(x > 1), which(x < 0))
x[i] <- 2 # to catch overflow errors in g(x)
fg <- g(x) / dcauchy(x)
theta.hat[3] <- mean(fg) * (1 - 0)
se[3] <- sd(fg)

u <- runif(m)
# f3, inverse transform method
x <- -log(1 - u * (1 - exp(-1)))
fg <- g(x) / (exp(-x) / (1 - exp(-1)))
theta.hat[4] <- mean(fg) * (1 - 0)
se[4] <- sd(fg)

u <- runif(m)
# f4, inverse transform method
x <- tan(pi * u / 4)
fg <- g(x) / (4 / ((1 + x^2) * pi)) # fg = f(x) / g(x)
theta.hat[5] <- mean(fg) * (1 - 0)
se[5] <- sd(fg)


# Stratified Sampling

Stratified Sampling là một phương pháp lấy mẫu được sử dụng để giảm sai số so với việc lấy mẫu ngẫu nhiên đơn giản. Trong Stratified Sampling, dữ liệu được chia thành các nhóm con gọi là strata, và mẫu được lấy một cách độc lập từ mỗi stratum. Mỗi stratum có thể có một tỷ lệ khác nhau trong dữ liệu tổng thể.

Đối với bài toán tính tích phân, chúng ta có thể áp dụng Stratified Sampling bằng cách chia khoảng [0, π33π​] thành các stratum và lấy mẫu ngẫu nhiên từ mỗi stratum. Sau đó, chúng ta tính toán giá trị của hàm sin⁡(t)sin(t) tại các điểm mẫu và tính trung bình của chúng để ước lượng tích phân.

Dưới đây là một triển khai của phương pháp này trong R để tính tích phân sin(x) từ 0 đến pi/3:

In [6]:
# Số lượng stratum
num_strata <- 10

# Số lượng điểm lấy mẫu trong mỗi stratum
samples_per_stratum <- 10000

# Tính chiều dài của mỗi stratum
stratum_length <- pi/3 / num_strata

# Khởi tạo biến để lưu trữ tổng giá trị hàm sin(t)
total_sum <- 0

# Lặp qua từng stratum
for (i in 1:num_strata) {
    # Tính biên dưới và biên trên của stratum
    lower_bound <- (i - 1) * stratum_length
    upper_bound <- i * stratum_length
    
    # Lấy mẫu ngẫu nhiên từ stratum hiện tại
    t_values <- runif(samples_per_stratum, min = lower_bound, max = upper_bound)
    
    # Tính giá trị của hàm sin(t) tại các điểm mẫu
    sin_values <- sin(t_values)
    
    # Tính tổng của giá trị hàm sin(t) trong stratum hiện tại và cộng vào tổng chung
    total_sum <- total_sum + sum(sin_values)
}

# Tính ước lượng của tích phân bằng cách chia tổng chung cho số lượng mẫu đã lấy và số stratum
stratified_estimate <- total_sum / (samples_per_stratum * num_strata)

# In ra kết quả ước lượng của tích phân
cat("Stratified Sampling Estimate:", stratified_estimate, "\n")

Stratified Sampling Estimate: 0.4774638 


## Ví dụ 6.12 

In [7]:
M <- 20
# number of replicates
T2 <- numeric(4)
estimates <- matrix(0, 10, 2)
g <- function(x) {
    exp(-x - log(1 + x^2)) * (x > 0) * (x < 1)
}
for (i in 1:10) {
    estimates[i, 1] <- mean(g(runif(M)))
    T2[1] <- mean(g(runif(M / 4, 0, .25)))
    T2[2] <- mean(g(runif(M / 4, .25, .5)))
    T2[3] <- mean(g(runif(M / 4, .5, .75)))
    T2[4] <- mean(g(runif(M / 4, .75, 1)))
    estimates[i, 2] <- mean(T2)
}
estimates


0.5461896,0.5372817
0.5587807,0.5161543
0.5531443,0.5323826
0.4784457,0.5253398
0.4868378,0.5337532
0.5070597,0.5246502
0.5537077,0.5159519
0.5202127,0.5391458
0.6007477,0.5416512
0.5523200,0.5027102


# Stratified Importance Sampling

Stratified Importance Sampling kết hợp hai phương pháp: Stratified Sampling và Importance Sampling. Trong phương pháp này, chúng ta chia dữ liệu thành các stratum như trong Stratified Sampling, nhưng thay vì lấy mẫu ngẫu nhiên từ mỗi stratum, chúng ta lấy mẫu sử dụng một phân phối quan trọng được chọn sao cho tối ưu cho từng stratum.

Dưới đây là một ví dụ code triển khai Stratified Importance Sampling trong R cho bài toán tính tích phân của hàm $f(x)=e^{−x}$ trên khoảng từ 0 đến 1:

In [8]:
# Số lượng stratum
num_strata <- 10

# Số lượng điểm lấy mẫu trong mỗi stratum
samples_per_stratum <- 10000

# Tính chiều dài của mỗi stratum
stratum_length <- 1 / num_strata

# Khởi tạo biến để lưu trữ tổng giá trị hàm f(x)
total_sum <- 0

# Lặp qua từng stratum
for (i in 1:num_strata) {
    # Tính biên dưới và biên trên của stratum
    lower_bound <- (i - 1) * stratum_length
    upper_bound <- i * stratum_length

    # Lấy mẫu ngẫu nhiên từ phân phối quan trọng (ví dụ: một phân phối chiều rộng hơn stratum)
    x_values <- rexp(samples_per_stratum, rate = 2) * stratum_length + lower_bound

    # Tính giá trị của hàm f(x) tại các điểm mẫu
    f_values <- exp(-x_values)

    # Tính trọng số quan trọng cho mỗi điểm mẫu
    importance_weights <- dexp(x_values - lower_bound, rate = 2) / stratum_length

    # Tính tổng của giá trị hàm f(x) với trọng số quan trọng và cộng vào tổng chung
    total_sum <- total_sum + sum(f_values * importance_weights)
}

# Tính ước lượng của tích phân bằng cách chia tổng chung cho số lượng mẫu đã lấy và số stratum
stratified_importance_estimate <- total_sum / (samples_per_stratum * num_strata)

# In ra kết quả ước lượng của tích phân
cat("Stratified Importance Sampling Estimate:", stratified_importance_estimate, "\n")


Stratified Importance Sampling Estimate: 11.54923 
